In [24]:
options(warn = -1)

In [22]:
library(lubridate)
library(rgdal)

Warning message:
"package 'rgdal' was built under R version 4.0.5"
Loading required package: sp

Warning message:
"package 'sp' was built under R version 4.0.5"
rgdal: version: 1.5-23, (SVN revision 1121)
Geospatial Data Abstraction Library extensions to R successfully loaded
Loaded GDAL runtime: GDAL 3.2.1, released 2020/12/29
Path to GDAL shared files: C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/gdal
GDAL binary built with GEOS: TRUE 
Loaded PROJ runtime: Rel. 7.2.1, January 1st, 2021, [PJ_VERSION: 721]
Path to PROJ shared files: C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/proj
PROJ CDN enabled: TRUE
Linking to sp version:1.4-5
To mute warnings of possible GDAL/OSR exportToProj4() degradation,
use options("rgdal_show_exportToProj4_warnings"="none") before loading rgdal.
Overwritten PROJ_LIB was C:/Users/clid1852/Documents/R/win-library/4.0/rgdal/proj



In [1]:
inpath <- 'T:/Data/COUNTS/Nonmotorized Counts/Summary Tables/Bicycle/'

In [2]:
rawdata <- read.csv(paste0(inpath, 'Bicycle_HourlyForTableau.csv'))

In [3]:
names(rawdata)

[1] "Direction"       "ObsHours"        "Location"        "LocationId"     
 [5] "Day"             "Month"           "MonthDesc"       "Year"           
 [9] "Date"            "Season"          "Weekday"         "IsHoliday"      
[13] "Period"          "DailyCounts"     "IsOneway"        "OnewayDirection"
[17] "IsSidewalk"      "UoInSession"     "IsSpecialEvent"  "UniqueId"       
[21] "Hour"            "Hourly_Count"

In [5]:
rawdata$Season <- ifelse(rawdata$MonthDesc == "September", "Fall", rawdata$Season)

In [30]:
year <- 2021

In [6]:
data <- rawdata[rawdata$Year == year,]

In [7]:
locdata <- read.csv("T:/Data/COUNTS/Nonmotorized Counts/Supporting Data/Supporting Bicycle Data/CountLocationInformation.csv")

In [10]:
names(locdata)

[1] "LocationId"           "CountType"            "Direction"           
 [4] "FacilityType"         "ArrowAngle"           "RoadWidth"           
 [7] "IsAutomatic"          "Location"             "HasData"             
[10] "City"                 "DoubleCountLocation"  "IsOneway"            
[13] "OnewayDirection"      "IsSidewalk"           "Latitude"            
[16] "Longitude"            "ImageFilePath"        "Site_Name"           
[19] "Location_Description" "TAZ"                  "Visual"

In [44]:
locvars <- c('City', 'Location', 'Latitude', 'Longitude', 'Site_Name', 
             'DoubleCountLocation', 'IsOneway', 'OnewayDirection', 
             'IsSidewalk')

In [9]:
dim(locdata)

[1] 205  21

In [12]:
length(unique(locdata$Location))

[1] 205

In [13]:
data$Date <- as.Date(data$Date, "%Y-%m-%d")

In [15]:
data$DOY <- yday(data$Date)

In [36]:
datavars <- names(data)[which(!(names(data) %in% c('IsOneway', 'OnewayDirection', 'IsSidewalk')))]

In [45]:
outdata <- merge(data[,datavars], locdata[,locvars], by = 'Location')

In [28]:
outpath <- "T:/MPO/Bike&Ped/BikeCounting/StoryMap/BikeCounts/Output"

In [16]:
year <- 2021

In [46]:
write.csv(outdata, paste0(outpath, "/Bike_Counts_", year, ".csv"), row.names = FALSE)

In [21]:
dim(outdata)

[1] 430200     30

In [25]:
MPOBound <- readOGR(dsn = "V:/Data/Transportation", layer="MPO_Bound")

OGR data source with driver: ESRI Shapefile 
Source: "V:\Data\Transportation", layer: "MPO_Bound"
with 1 features
It has 3 fields


In [26]:
# require MPOBound
df2spdf <- function(df, lon_col_name, lat_col_name, trans = TRUE){
  lonlat <- sp::CRS("+proj=longlat +datum=WGS84 +ellps=WGS84 +towgs84=0,0,0")
  lon_col_no <- which(names(df)==lon_col_name)
  lat_col_no <- which(names(df)==lat_col_name)
  xy <- data.frame(df[,c(lon_col_no,lat_col_no)])
  coordinates(xy) <- c(lon_col_name, lat_col_name)
  proj4string(xy) <- lonlat
  spdf <- sp::SpatialPointsDataFrame(coords = xy, data = df)
  if(trans){
    spdf <- spTransform(spdf, CRS(proj4string(MPOBound)))
  }
  return(spdf)
}

In [47]:
outspdf <- df2spdf(outdata, 'Longitude', 'Latitude')

In [48]:
writeOGR(outspdf, dsn=outpath, layer=paste0("Bike_Counts_", year), 
         driver="ESRI Shapefile", overwrite_layer=TRUE)

In [42]:
names(outspdf)

[1] "Location"            "Direction"           "ObsHours"           
 [4] "LocationId"          "Day"                 "Month"              
 [7] "MonthDesc"           "Year"                "Date"               
[10] "Season"              "Weekday"             "IsHoliday"          
[13] "Period"              "DailyCounts"         "UoInSession"        
[16] "IsSpecialEvent"      "UniqueId"            "Hour"               
[19] "Hourly_Count"        "DOY"                 "Latitude"           
[22] "Longitude"           "Site_Name"           "DoubleCountLocation"
[25] "IsOneway"            "OnewayDirection"     "IsSidewalk"